# Grouping Data & Aggregating Variables:

1. Importing libraries and dataset
2. Finding the aggregated mean of the “order_number” column grouped by “department_id”
3. Creating a loyalty flag for existing customers using the transform() and loc() functions
4. Exploring spending habits of customer for marketing team
5. Categorising the different types of spenders to target via marketing campaigns
6. Determine frequent versus non-frequent customers in order to send relevant notifications to users within the app
7. Exporting the dataframe

## 1. Importing libraries and dataset

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Accessing EnvFile for path

%run EnvFile.ipynb

Stored 'path' (str)


In [3]:
# Importing dataframe

df_ords_prods_dept = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_dept_newcolumns.pkl'))

## 2. Finding the aggregated mean of the “order_number” column grouped by “department_id”

In [4]:
df_ords_prods_dept.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## 3. Creating a loyalty flag for existing customers using the transform() and loc() functions

In [5]:
# Split the data (user_id) into groups by .groupby() and assign new column (max_order) with aggregate values (max) of order_number using .transform()

df_ords_prods_dept['max_order'] = df_ords_prods_dept.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
# Checking the newly created column using index and head()

df_ords_prods_dept[['user_id', 'order_number', 'max_order']].head(60)

,user_id,order_number,max_order
0,1,1,10
1,1,2,10
2,1,3,10
3,1,4,10
4,1,5,10
5,1,6,10
6,1,7,10
7,1,8,10
8,1,9,10
9,1,10,10


In [7]:
# Creating loyal customer condition using .loc()

df_ords_prods_dept.loc[df_ords_prods_dept['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'

In [8]:
# Creating regular customer condition using .loc()

df_ords_prods_dept.loc[(df_ords_prods_dept['max_order'] <= 40) & (df_ords_prods_dept['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'

In [9]:
# Creating new customer condition using .loc()

df_ords_prods_dept.loc[df_ords_prods_dept['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [10]:
# Checking the frequency of new flag column using .value_counts()

df_ords_prods_dept['loyalty_flag'].value_counts(dropna = False)

Regular Customer    15876441
Loyal Customer      10284025
New Customer         6243823
Name: loyalty_flag, dtype: int64

In [11]:
# Checking the specific column results using index and head()

df_ords_prods_dept[['user_id', 'order_number', 'max_order', 'loyalty_flag']].head(15)

,user_id,order_number,max_order,loyalty_flag
0,1,1,10,New Customer
1,1,2,10,New Customer
2,1,3,10,New Customer
3,1,4,10,New Customer
4,1,5,10,New Customer
5,1,6,10,New Customer
6,1,7,10,New Customer
7,1,8,10,New Customer
8,1,9,10,New Customer
9,1,10,10,New Customer


## 4. Exploring spending habits of customer for marketing team

In [12]:
# Split the data (loyalty_flag) into groups using groupby() and apply aggregate functions using agg() to obtain basic statistics (mean, min, max values) of prices

df_ords_prods_dept.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal Customer    7.774488  1.0  25.0
New Customer      7.802479  1.0  25.0
Regular Customer  7.799419  1.0  25.0

#### The prices of products purchased by loyal customers do not differ hugely from those purchased by regular or new customers.

## 5. Categorising the different types of spenders to target via marketing campaigns

Creating a spending flag for each user based on the average price across all their orders using the following criteria:

- If the mean of the prices of products purchased by a user is lower than 10, then flagging them as a “Low Spender.”
- If the mean of the prices of products purchased by a user is higher than or equal to 10, then flagging them as a “High Spender.”

In [13]:
# Split the data (user_id) into groups by .groupby() and assign new column (mean_price) with aggregate values (max) of prices using .transform()

df_ords_prods_dept['average_price'] = df_ords_prods_dept.groupby(['user_id'])['prices'].transform(np.mean)

In [14]:
# Checking the newly created column using .head()

df_ords_prods_dept[['user_id', 'prices', 'average_price']].head(15)

,user_id,prices,average_price
0,1,9.0,6.367797
1,1,9.0,6.367797
2,1,9.0,6.367797
3,1,9.0,6.367797
4,1,9.0,6.367797
5,1,9.0,6.367797
6,1,9.0,6.367797
7,1,9.0,6.367797
8,1,9.0,6.367797
9,1,9.0,6.367797


In [15]:
# Creating low spender condition using .loc()

df_ords_prods_dept.loc[df_ords_prods_dept['average_price'] < 10, 'spending_flag'] = 'Low Spender'

In [16]:
# Creating high spender condition using .loc()

df_ords_prods_dept.loc[df_ords_prods_dept['average_price'] >= 10, 'spending_flag'] = 'High Spender'

In [17]:
# Checking the frequency of new flag column using .value_counts()

df_ords_prods_dept['spending_flag'].value_counts(dropna = False)

Low Spender     32284330
High Spender      119959
Name: spending_flag, dtype: int64

In [18]:
# Checking the specific column results using index and head()

df_ords_prods_dept[['user_id', 'prices', 'average_price', 'spending_flag']].head(15)

,user_id,prices,average_price,spending_flag
0,1,9.0,6.367797,Low Spender
1,1,9.0,6.367797,Low Spender
2,1,9.0,6.367797,Low Spender
3,1,9.0,6.367797,Low Spender
4,1,9.0,6.367797,Low Spender
5,1,9.0,6.367797,Low Spender
6,1,9.0,6.367797,Low Spender
7,1,9.0,6.367797,Low Spender
8,1,9.0,6.367797,Low Spender
9,1,9.0,6.367797,Low Spender


## 6. Determine frequent versus non-frequent customers in order to send relevant notifications to users within the app

Creating an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag will be as follows:
- If the median of “days_since_prior_order” is higher than 20, then the customer will be labeled a “Non-frequent Customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer will be labeled a “Regular Customer.”
- If the median is lower than or equal to 10, then the customer will be labeled a “Frequent Customer.”

In [19]:
# Split the data (user_id) into groups by .groupby() and assign new column (median_days_since_prior_order) with aggregate values (median) of days_since_prior_order using .transform()

df_ords_prods_dept['median_days_since_prior_order'] = df_ords_prods_dept.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
# Checking the newly created column using index and .head()

df_ords_prods_dept[['user_id', 'days_since_prior_order', 'median_days_since_prior_order']].head(15)

,user_id,days_since_prior_order,median_days_since_prior_order
0,1,NaN,20.5
1,1,15.0,20.5
2,1,21.0,20.5
3,1,29.0,20.5
4,1,28.0,20.5
5,1,19.0,20.5
6,1,20.0,20.5
7,1,14.0,20.5
8,1,0.0,20.5
9,1,30.0,20.5


In [21]:
# Creating non-frequent customer condition using .loc()

df_ords_prods_dept.loc[df_ords_prods_dept['median_days_since_prior_order'] > 20, 'order_frequency_flag'] = 'Non-frequent Customer'

In [22]:
# Creating regular customer condition using .loc()

df_ords_prods_dept.loc[(df_ords_prods_dept['median_days_since_prior_order'] > 10) & (df_ords_prods_dept['median_days_since_prior_order'] <= 20), 'order_frequency_flag'] = 'Regular Customer'

In [23]:
# Creating frequent customer condition using .loc()

df_ords_prods_dept.loc[df_ords_prods_dept['median_days_since_prior_order'] <= 10, 'order_frequency_flag'] = 'Frequent Customer'

In [24]:
# Checking the frequency of new flag column using .value_counts()

df_ords_prods_dept['order_frequency_flag'].value_counts(dropna = False)

Frequent Customer        21559233
Regular Customer          7208688
Non-frequent Customer     3636363
NaN                             5
Name: order_frequency_flag, dtype: int64

In [25]:
# Checking the specific column results using index and head()

df_ords_prods_dept[['user_id', 'days_since_prior_order', 'median_days_since_prior_order', 'order_frequency_flag']].head(15)

,user_id,days_since_prior_order,median_days_since_prior_order,order_frequency_flag
0,1,NaN,20.5,Non-frequent Customer
1,1,15.0,20.5,Non-frequent Customer
2,1,21.0,20.5,Non-frequent Customer
3,1,29.0,20.5,Non-frequent Customer
4,1,28.0,20.5,Non-frequent Customer
5,1,19.0,20.5,Non-frequent Customer
6,1,20.0,20.5,Non-frequent Customer
7,1,14.0,20.5,Non-frequent Customer
8,1,0.0,20.5,Non-frequent Customer
9,1,30.0,20.5,Non-frequent Customer


## 7. Exporting the dataframe

In [26]:
# Exporting the updated df with new aggregated columns

df_ords_prods_dept.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_dept_aggcolumns.pkl'))